<a href="https://colab.research.google.com/github/seungyeon38/Webtoon_recommendation_system/blob/master/CrawlingAndDataFileDownload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# from google.colab import drive
# drive.mount('/content/drive')

In [30]:
import numpy as np
import pandas as pd
import string
string.punctuation
from math import log
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
import re
from tqdm import tqdm
from gensim.summarization import keywords
from textblob import TextBlob

!pip install gensim


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [31]:
comic = pd.read_csv('/content/sample_data/Webtoon Dataset.csv')

In [32]:
comic.head(3)

,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...
1,1,True Beauty,Yaongyi,39.9M,Romance,9.60,6.4M,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...
2,2,Midnight Poppy Land,Lilydusk,10.4M,Romance,9.81,2.1M,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...


In [33]:
!pip install beautifulsoup4


import requests
from bs4 import BeautifulSoup

def extract_similar_webtoons(url): 
# url = 'https://www.webtoons.com/en/romance/age-matters/list?title_no=1364'
  response = requests.get(url)
  similar_webtoon_lists = []
  if response.status_code == 200:
      html = response.text
      soup = BeautifulSoup(html, 'html.parser')
      title = soup.select_one('#content > div.cont_box > div.detail_other.NE\=a\:enw > ul')
      if title != None :
        title_list = title.select('li > a > div.info_area > p.subj')
        for title in title_list:
          similar_webtoon_lists.append(title.get_text())
      elif title == None : 
        print("None")

      print(similar_webtoon_lists)
  else : 
      print(response.status_code)

  return similar_webtoon_lists


comic['Similar_Webtoons'] = comic['Reading Link'].apply(lambda url: extract_similar_webtoons(url))


['Lore Olympus', 'Age Matters', 'KILLMAX']
['SAVE ME', 'Edith', 'KILLMAX']
['SubZero', 'Unlovable Replacement', 'KILLMAX']
["Let's Play", 'True Beauty', 'Freaking Romance']
['unTouchable', 'Devil Number 4', 'KILLMAX']
['I Love Yoo', 'Fluffy Boyfriend', 'Strawberry Seafoam']
[]
['Weak Hero', 'The World is Money and Power', 'Get Schooled']
['Cheese in the Trap', "Let's Play", 'The Four of Them']
['Unlovable Replacement', 'Version Day and Night', 'Midnight Rain']
['True Beauty', 'Back to You', 'To Love Your Enemy']
[]
['Your Smile is a Trap', 'To Love Your Enemy', 'Seasons of Blossom']
[]
['Jungle Juice', 'Dungeons & Artifacts', 'Hero Killer']
['Cheese in the Trap', 'True Beauty', 'Devil Number 4']
['Act Like You Love Me!', 'Let Me Stay Over Tonight!', 'Love Falls Like Rain']
['The God of High School', 'unOrdinary', 'Lookism']
['Distant Sky', "It's Mine", 'Rotten']
[]
[]
['Viral Hit', 'Pyramid Game', 'The Build Up']
['unOrdinary', 'Oh! Holy', 'I Love Yoo']
['Your Throne', 'Men of the Hare

In [34]:
# 축약형 
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "she's": "she is"}

def regularize_contractions(text):

    contractionfree_list = []
    
    for word in text.split(" "):
      if word in contractions.keys():
        contractionfree_list.append(contractions[word])
      else :
        contractionfree_list.append(word)
      
    return " ".join(contractionfree_list)


In [35]:
#defining the function to remove punctuation
# 글자가 puctuation에 해당하면 " "를, 해당하지 않으면 글자 그대로

def remove_punctuation(text):
    punctuationfree_list = []

    for word in text:
      if word in string.punctuation:
        punctuationfree_list.append(" ")
      else :
        punctuationfree_list.append(word)
    return "".join(punctuationfree_list)


# def remove_punctuation(text):
#     punctuationfree = "".join([i for i in text if i not in string.punctuation])
#     return punctuationfree

In [36]:
# #defining function for tokenization
# 숫자 포함 토큰화
# def tokenization(text):
#     tokens = re.split('\W+', text)

#     return tokens

# 영어만 단어단위로 토큰화
p = re.compile('[a-z]+')

def tokenization(text):
    # print(text)
    result = p.findall(text)
    # print(result)
    return result

In [37]:
#stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
# print(stopwords[:20]) 

#defining the function to remove stopwords from tokenized text
# stopwords 제거 
def remove_stopwords(text):
    for word in text:
      output= [word for word in text if word not in stopwords]
    return output

In [38]:
#setting lower case
comic['Lower_Summary'] = comic['Summary'].apply(lambda x: x.lower())
comic['Lower_Summary'].head(3)

0    she's young, single and about to achieve her d...
1    after binge-watching beauty videos online, a s...
2    after making a grisly discovery in the country...
Name: Lower_Summary, dtype: object

In [39]:
comic['No_Contraction_Summary'] = comic['Lower_Summary'].apply(lambda x:regularize_contractions(x))
comic['No_Contraction_Summary'].head(3)

0    she is young, single and about to achieve her ...
1    after binge-watching beauty videos online, a s...
2    after making a grisly discovery in the country...
Name: No_Contraction_Summary, dtype: object

In [40]:
comic['Clean_Summary'] = comic['No_Contraction_Summary'].apply(lambda x:remove_punctuation(x))
comic['Clean_Summary'].head(3)

0    she is young  single and about to achieve her ...
1    after binge watching beauty videos online  a s...
2    after making a grisly discovery in the country...
Name: Clean_Summary, dtype: object

In [41]:
#applying function to the column
comic['Tokenied_Summary'] = comic['Clean_Summary'].apply(lambda x: tokenization(x))

comic['Tokenied_Summary'].head(3)

0    [she, is, young, single, and, about, to, achie...
1    [after, binge, watching, beauty, videos, onlin...
2    [after, making, a, grisly, discovery, in, the,...
Name: Tokenied_Summary, dtype: object

In [42]:
#applying the function
comic['No_Stopwords_Summary'] = comic['Tokenied_Summary'].apply(lambda x:remove_stopwords(x))

comic['No_Stopwords_Summary'].head(3)

0    [young, single, achieve, dream, creating, incr...
1    [binge, watching, beauty, videos, online, shy,...
2    [making, grisly, discovery, countryside, small...
Name: No_Stopwords_Summary, dtype: object

In [43]:
# def extract_noun(text):
#   part_of_speech_text = pos_tag(text)
#   result = []
#   for word in part_of_speech_text:
#     if !(word[1] == "NN" or word[1] == "NNS"):
#         result.append(word[0])

#   return result; 

# comic['Noun_Summary'] = comic['No_Stopwords_Summary'].apply(lambda x:extract_noun(x))

# comic['Noun_Summary'].head(3)

In [44]:
 # stemmer = nltk.stem.SnowballStemmer('english')


 # def stem(words_list):
 # stemmer_words = [stemmer.stem(word) for word in words_list]

 # return stemmer_words

# comic['Stemmed_Summary'] = comic['No_Stopwords_Summary'].apply(lambda x:stem(x))
# comic['Stemmed_Summary'].head(3)



In [45]:
# df1 = comic['No_Stopwords_Summary']
# doc_words_list = df1.values.tolist()

# print(doc_words_list)


def wordlist_to_string(words_list) :
  doc = []
  for doc_word in words_list :
    doc.append(doc_word)
  return " ".join(doc)


#comic['Summary_final'] = comic['Stemmed_Summary'].apply(lambda x:wordlist_to_string(x))
comic['Summary_final'] = comic['No_Stopwords_Summary'].apply(lambda x:wordlist_to_string(x))

comic['Summary_final'].head(3)

0    young single achieve dream creating incredible...
1    binge watching beauty videos online shy comic ...
2    making grisly discovery countryside small town...
Name: Summary_final, dtype: object

In [46]:
def extract_keywords(doc):
  keyword_list = []
  keyword_string = keywords(doc)
  keyword_list = keyword_string.split('\n')

  return keyword_list;


comic['Summary_Keywords'] = comic['Summary_final'].apply(lambda x:extract_keywords(x))

comic.head(3)

,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link,Similar_Webtoons,Lower_Summary,No_Contraction_Summary,Clean_Summary,Tokenied_Summary,No_Stopwords_Summary,Summary_final,Summary_Keywords
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...,"[Lore Olympus, Age Matters, KILLMAX]","she's young, single and about to achieve her d...","she is young, single and about to achieve her ...",she is young single and about to achieve her ...,"[she, is, young, single, and, about, to, achie...","[young, single, achieve, dream, creating, incr...",young single achieve dream creating incredible...,"[game, gaming, popular streamer, single, life,..."
1,1,True Beauty,Yaongyi,39.9M,Romance,9.60,6.4M,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...,"[SAVE ME, Edith, KILLMAX]","after binge-watching beauty videos online, a s...","after binge-watching beauty videos online, a s...",after binge watching beauty videos online a s...,"[after, binge, watching, beauty, videos, onlin...","[binge, watching, beauty, videos, online, shy,...",binge watching beauty videos online shy comic ...,"[secret, lived, watching, standing, prettiest ..."
2,2,Midnight Poppy Land,Lilydusk,10.4M,Romance,9.81,2.1M,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...,"[SubZero, Unlovable Replacement, KILLMAX]",after making a grisly discovery in the country...,after making a grisly discovery in the country...,after making a grisly discovery in the country...,"[after, making, a, grisly, discovery, in, the,...","[making, grisly, discovery, countryside, small...",making grisly discovery countryside small town...,"[underworld, takes, intimidating, grisly]"


In [47]:
comic.to_csv('webtoon_data.csv')

In [48]:
comic_df = comic[['id','Name', 'Writer', 'Genre', 'Rating',
                 'Summary_Keywords']]
comic_df.head(1)

,id,Name,Writer,Genre,Rating,Summary_Keywords
0,0,Let's Play,Leeanne M. Krecic (Mongie),Romance,9.62,"[game, gaming, popular streamer, single, life,..."


In [49]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# make keywords to type of string for analyze
Keywords = comic['Summary_Keywords'].apply(lambda x:wordlist_to_string(x))

# create function to get polarity score
def polarity(text):
    senti_analyzer = SentimentIntensityAnalyzer()
    senti_scores = senti_analyzer.polarity_scores(text)

    return senti_scores

# output of polarity score
comic_df['Polarity'] = Keywords.apply(polarity)
comic_df.head(2)

# output of sentiment type
comic_df['compound'] = comic_df['Polarity'].apply(lambda score_dict: score_dict['compound']) 
comic_df['sentiment_type']='' 
comic_df.loc[comic_df.compound > 0,'sentiment_type']='POSITIVE' 
comic_df.loc[comic_df.compound == 0,'sentiment_type']='NEUTRAL' 
comic_df.loc[comic_df.compound < 0,'sentiment_type']='NEGATIVE'
comic_df.head(5)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/

,id,Name,Writer,Genre,Rating,Summary_Keywords,Polarity,compound,sentiment_type
0,0,Let's Play,Leeanne M. Krecic (Mongie),Romance,9.62,"[game, gaming, popular streamer, single, life,...","{'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'comp...",0.4215,POSITIVE
1,1,True Beauty,Yaongyi,Romance,9.60,"[secret, lived, watching, standing, prettiest ...","{'neg': 0.0, 'neu': 0.367, 'pos': 0.633, 'comp...",0.7845,POSITIVE
2,2,Midnight Poppy Land,Lilydusk,Romance,9.81,"[underworld, takes, intimidating, grisly]","{'neg': 0.492, 'neu': 0.508, 'pos': 0.0, 'comp...",-0.4404,NEGATIVE
3,3,Age Matters,Enjelicious,Romance,9.79,"[happy, rules, way]","{'neg': 0.0, 'neu': 0.351, 'pos': 0.649, 'comp...",0.5719,POSITIVE
4,4,Unholy Blood,Lina Im / Jeonghyeon Kim,Supernatural,9.85,"[hayan forced, force, destroy chance normal]","{'neg': 0.565, 'neu': 0.261, 'pos': 0.174, 'co...",-0.6705,NEGATIVE


In [50]:
from sklearn.feature_extraction.text import CountVectorizer

comic_df['Summary_literal'] = comic_df['Summary_Keywords'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
Summary_mat = count_vect.fit_transform(comic_df['Summary_literal'])
print(Summary_mat.shape)

(569, 3808)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [51]:
from sklearn.metrics.pairwise import cosine_similarity

Summary_sim = cosine_similarity(Summary_mat, Summary_mat)
print(Summary_sim.shape)
print(Summary_sim[:2])

(569, 569)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [52]:
# 0번 레코드의 경우 94번 레코드와 가장 유사도 높고, 284번 레코드와 가장 유사도 낮음을 나타내는것

Summary_sim_sorted_ind = Summary_sim.argsort()[:, ::-1]
print(Summary_sim_sorted_ind[:1])

[[  0 415 135 260  23 490 239 443 248 319 326 191 355 250 107  18  94 230
    8  76  92 329  37 351  58 410  97 401 174 179 262  15 509 264 213  85
   99  77  45 188 210  98 125 211 354 411  60 238 186 177 266 178 265 180
  181 182 183 184 185 187 175 263 189 190 261 272 192 193 194 195 196 197
  176 267 259 268 152 153 275 154 155 156 157 158 159 269 160 161 162 163
  164 165 274 166 167 168 169 170 171 172 173 198 257 258 244 219 220 221
  222 223 224 225 226 227 246 245 228 237 229 243 242 231 232 233 234 235
  241 236 240 218 217 216 215 199 200 273 201 256 202 203 204 255 205 254
  206 253 207 208 209 252 270 251 249 212 247 214 271 141 151  54  39  40
   41  42  43  44  46  47  48  49  50  51  52  53  55  73  56  57  59  61
   62  63  64  65  66  67  68  69  70  71  38  36  35  34   1   2   3   4
    5   6   7   9  10  11  12  13  14  16  17  19  20  21  22  24  25  26
   27  28  29  30  31  32  33  72  74 150 133 118 119 120 121 122 123 124
  126 127 128 129 130 131 132 134  75 

In [53]:
def find_sim_comic(df, sorted_ind, c_name, top_n=10):
    comic_name = df[df['Name'] == c_name]
    comic_index = comic_name.index.values
    similar_indexes = sorted_ind[comic_index, :(top_n*5)]
    similar_indexes = similar_indexes.reshape(-1)
    print(similar_indexes)
    return df.iloc[similar_indexes][:top_n]
    # return df.iloc[similar_indexes].sort_values('Rating', ascending=False)[:top_n] 평점순으로

In [54]:
# 'Death of a Pop Star'와 비슷한 키워드 가진 웹툰 10개 추천
similar_comic = find_sim_comic(comic_df, Summary_sim_sorted_ind, 'Death of a Pop Star',10)
similar_comic[['Name', 'Rating' , 'Summary_Keywords']]

[ 30 104   2  80 220 545 101 277 249 183 182 184 192 181 185 186 187 188
 189 190 191 195 193 202 208 207 206 205 204 203 201 194 200 199 198 197
 196 179 180 568 178 153 159 158 157 156 155 154 152 161]


,Name,Rating,Summary_Keywords
30,Death of a Pop Star,9.34,"[takes, lucky, reaper, dead]"
104,Oh! Holy,9.70,"[invisible, takes, school, remain]"
2,Midnight Poppy Land,9.81,"[underworld, takes, intimidating, grisly]"
80,The Horizon,9.84,"[dead bodies, event wiped]"
220,When Jasy Whistles,9.76,"[old, times, asuncion, jasy, dead]"
545,Rotten,9.68,"[evil, sees dead people officer]"
101,Gourmet Hound,9.82,"[taste, loved years lucky accident, woman]"
277,Rise from Ashes,9.70,"[ghosts, ghost, dead, world, abandoned house, ..."
249,Nomads,9.85,"[nomads, nomad, turn, turns, starts, start, sp..."
183,The God of High School,9.70,"[high school, plot, god, began]"
